In [121]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.corpus import wordnet
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet as wn

In [122]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jeleez\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [123]:
# CSV 파일에서 데이터 불러오기
apps_data = pd.read_csv('cleaned_apps_data.csv')

In [124]:
# 2. 앱 이름을 불용어 리스트로 처리
app_names = apps_data['Name'].apply(lambda x: x.lower()).tolist()  # 앱 이름을 소문자로 변환하여 리스트로 수집


In [125]:
# 3. 추가적인 불용어 정의
additional_stopwords = ['pdf', 'file', 'app', 'lite', 'application']

In [126]:
# 4. 기본 불용어와 추가 불용어 결합
stop_words = list(text.ENGLISH_STOP_WORDS.union(app_names).union(additional_stopwords))


In [127]:
# 2. Sentence-BERT 모델 로드 및 문장 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')  # Sentence-BERT 모델
embeddings = model.encode(apps_data['Cleaned_Description'].values)  # 앱 설명에 대한 임베딩 생성


c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [128]:
# 3. K-Means 클러스터링 수행
n_clusters = 5  # 클러스터 수 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)


c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [129]:
# 4. 클러스터 결과를 데이터프레임에 추가
apps_data['Cluster'] = clusters



In [81]:
# 7. TF-IDF 기반 주요 키워드 추출 함수
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=1000)
tfidf_matrix = vectorizer.fit_transform(apps_data['Cleaned_Description'])

def get_top_keywords(tfidf_matrix, clusters, top_n=5):
    cluster_centers = np.zeros((np.unique(clusters).size, tfidf_matrix.shape[1]))
    
    for cluster in np.unique(clusters):
        cluster_centers[cluster] = tfidf_matrix[clusters == cluster].mean(axis=0)
    
    terms = vectorizer.get_feature_names_out()
    top_keywords = []
    
    for cluster in range(cluster_centers.shape[0]):
        center = cluster_centers[cluster]
        top_indices = center.argsort()[::-1][:top_n]
        keywords = [terms[i] for i in top_indices]
        top_keywords.append(keywords)
    
    return top_keywords

c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['05', '09', '10', '10in1', '125', '152', '153', '164', '2024', '2025', '252', '264', '309', '4x', '51', '7000', 'acro', 'adblocker', 'ads', 'agoda', 'ai', 'air', 'airplay', 'airreceiver', 'aliexpress', 'alpinequest', 'analog', 'anatomy', 'ancient', 'annotations', 'apparel', 'arar', 'assen', 'astro', 'atlas', 'audio', 'band', 'bethlehem', 'bible', 'bike', 'billionaire', 'blocker', 'book', 'booster', 'boy', 'breitling', 'bs', 'buy', 'camera', 'capcut', 'car', 'carrera', 'cast', 'celest1710', 'chak', 'cheap', 'cherieviewer', 'classic', 'clock', 'codec', 'com', 'comic', 'comicviewer', 'counter', 'coupang', 'creator', 'd036', 'd182', 'ddmusic', 'delivery', 'delta', 'dict', 'digi', 'digital', 'disney', 'divx', 'dlna', 'document', 'documents', 'docx', 'dosbox', 'dra

In [82]:
# 6. 클러스터별 상위 5개 키워드 추출
top_keywords_per_cluster = get_top_keywords(tfidf_matrix, clusters, top_n=5)



In [83]:
# 7. 키워드 기반으로 상위 카테고리 이름 생성
def generate_category_name(keywords):
    return " & ".join(keywords[:2])  # 상위 2개의 키워드를 연결하여 이름 생성



In [84]:
# 9. WordNet을 사용하여 키워드를 일반화하는 함수
from nltk.corpus import wordnet

In [85]:
def generalize_word(word):
    synsets = wordnet.synsets(word)
    if synsets:
        # 첫 번째 동의어 집합을 사용하여 일반화
        hypernyms = synsets[0].hypernyms()
        if hypernyms:
            return hypernyms[0].lemmas()[0].name()  # 가장 일반적인 상위 단어 반환
    return word  # 동의어 집합이 없으면 원래 단어 반환

In [86]:
# 10. 키워드 기반으로 상위 카테고리 이름 생성
def generate_category_name(keywords):
    generalized_keywords = [generalize_word(kw) for kw in keywords]
    return " & ".join(generalized_keywords[:2])  # 상위 2개의 일반화된 키워드를 연결

In [87]:
# 11. 클러스터별 상위 카테고리 이름 생성 및 출력
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    print(f"Cluster {cluster_num} Name: {category_name}")


Cluster 0 Name: timepiece & external_body_part
Cluster 1 Name: paper & mobile
Cluster 2 Name: commodity & property
Cluster 3 Name: record & scholar
Cluster 4 Name: auditory_communication & visual_communication


In [88]:
# 12. 클러스터 이름을 데이터프레임에 추가
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    apps_data.loc[apps_data['Cluster'] == cluster_num, 'Super_Category'] = category_name


In [89]:
# 10. 결과 확인
print(apps_data[['Name', 'Category', 'Cleaned_Description', 'Cluster', 'Super_Category']])

                        Name          Category  \
0                 PDF Reader             Tools   
1                  모바일 건강보험증  Health & Fitness   
2       All Documents Viewer      Productivity   
3      A.(에이닷) - 나만의 AI 개인비서         Lifestyle   
4                     온누리상품권           Finance   
..                       ...               ...   
190              OruxMaps GP    Travel & Local   
191           굴삭기 면허시험 시뮬레이터         Education   
192  EVERYDAY 10-Hands Style   Personalization   
193               Astro Gold         Lifestyle   
194  ACRO Classic Wonder T50   Personalization   

                                   Cleaned_Description  Cluster  \
0    PDF app Android best viewer viewing PDF PDF Re...        3   
1    1 mobile health insurance card app public serv...        1   
2    features include View Multiple Types Files Sca...        3   
3    Adot newly evolved AI personal assistant AI pe...        4   
4    Onnuri Gift Certificate service allows conveni...        1 

In [90]:
# clustering 'clustered_apps_data.csv' 파일로 저장
apps_data.to_csv('clustered_apps_data.csv', index=False)

In [91]:
# clustering 'clustered_apps_data.csv' 파일로 저장
apps_data.to_excel('clustered_apps_data.xlsx', index=False)

# WordNet

In [105]:
# 2. WordNet을 사용하여 키워드의 상위 개념을 찾는 함수
def get_hypernyms(word):
    synsets = wn.synsets(word)
    if synsets:
        hypernyms = synsets[0].hypernyms()
        if hypernyms:
            return hypernyms[0].lemmas()[0].name()  # 첫 번째 상위 개념 반환
    return word  # 상위 개념이 없으면 원래 단어 반환

In [106]:
# 3. 키워드를 상위 개념으로 변환하는 함수
def generalize_keywords(keywords):
    generalized_keywords = [get_hypernyms(word) for word in keywords]
    return generalized_keywords


In [107]:
# 4. Sentence-BERT 모델 로드 및 문장 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(apps_data['Description'].values)


c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [108]:
# 5. K-Means 클러스터링 수행
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)

c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [115]:
# 7. TF-IDF 기반 주요 키워드 추출
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(apps_data['Cleaned_Description'])

def get_top_keywords(tfidf_matrix, clusters, top_n=5):
    cluster_centers = np.zeros((np.unique(clusters).size, tfidf_matrix.shape[1]))
    
    for cluster in np.unique(clusters):
        cluster_centers[cluster] = tfidf_matrix[clusters == cluster].mean(axis=0)
    
    terms = vectorizer.get_feature_names_out()
    top_keywords = []
    
    for cluster in range(cluster_centers.shape[0]):
        center = cluster_centers[cluster]
        top_indices = center.argsort()[::-1][:top_n]
        keywords = [terms[i] for i in top_indices]
        top_keywords.append(keywords)
    
    return top_keywords

In [116]:
# 8. 클러스터별 상위 5개 키워드 추출
top_keywords_per_cluster = get_top_keywords(tfidf_matrix, clusters, top_n=5)

In [117]:
# 9. 상위 개념 기반 카테고리 이름 생성
def generate_category_name(keywords):
    generalized_keywords = generalize_keywords(keywords)
    return " & ".join(set(generalized_keywords[:2]))  # 중복되지 않은 상위 2개의 단어

In [118]:
# 10. 클러스터별 상위 카테고리 이름 생성 및 출력
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    print(f"Cluster {cluster_num} Name: {category_name}")


Cluster 0 Name: app & representation
Cluster 1 Name: pdf & record
Cluster 2 Name: timepiece & external_body_part
Cluster 3 Name: visual_communication & auditory_communication
Cluster 4 Name: mobile & paper


# Cosine Similarity

In [130]:
# 5. TF-IDF 기반 주요 키워드 추출
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(apps_data['Cleaned_Description'])

def get_top_keywords(tfidf_matrix, clusters, top_n=5):
    cluster_centers = np.zeros((np.unique(clusters).size, tfidf_matrix.shape[1]))
    
    for cluster in np.unique(clusters):
        cluster_centers[cluster] = tfidf_matrix[clusters == cluster].mean(axis=0)
    
    terms = vectorizer.get_feature_names_out()
    top_keywords = []
    
    for cluster in range(cluster_centers.shape[0]):
        center = cluster_centers[cluster]
        top_indices = center.argsort()[::-1][:top_n]
        keywords = [terms[i] for i in top_indices]
        top_keywords.append(keywords)
    
    return top_keywords

In [131]:
# 6. 클러스터별 상위 5개 키워드 추출
top_keywords_per_cluster = get_top_keywords(tfidf_matrix, clusters, top_n=5)

In [132]:
# 7. 각 키워드를 임베딩하여 벡터 생성
def get_keyword_embedding(keywords):
    return model.encode(keywords)

In [133]:
# 8. Cosine Similarity를 사용하여 중심 단어 선택
def select_representative_word(keywords):
    keyword_embeddings = get_keyword_embedding(keywords)
    similarity_matrix = cosine_similarity(keyword_embeddings)

    # 각 단어의 유사도 합 계산
    similarity_sums = similarity_matrix.sum(axis=1)
    
    # 유사도 합이 가장 큰 단어를 대표 단어로 선택
    representative_idx = np.argmax(similarity_sums)
    return keywords[representative_idx]

In [134]:
# 9. 클러스터별 대표 단어 생성
def generate_category_name(keywords):
    return select_representative_word(keywords)

In [135]:
# 10. 클러스터별 대표 단어 생성 및 출력
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    print(f"Cluster {cluster_num} Name: {category_name}")


Cluster 0 Name: face
Cluster 1 Name: payment
Cluster 2 Name: products
Cluster 3 Name: reader
Cluster 4 Name: videos


In [136]:
# 11. 클러스터 이름을 데이터프레임에 추가
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    apps_data.loc[apps_data['Cluster'] == cluster_num, 'Super_Category'] = category_name


In [137]:
# 결과 확인
print(apps_data[['Name', 'Category', 'Description', 'Cluster', 'Super_Category']])


                        Name          Category  \
0                 PDF Reader             Tools   
1                  모바일 건강보험증  Health & Fitness   
2       All Documents Viewer      Productivity   
3      A.(에이닷) - 나만의 AI 개인비서         Lifestyle   
4                     온누리상품권           Finance   
..                       ...               ...   
190              OruxMaps GP    Travel & Local   
191           굴삭기 면허시험 시뮬레이터         Education   
192  EVERYDAY 10-Hands Style   Personalization   
193               Astro Gold         Lifestyle   
194  ACRO Classic Wonder T50   Personalization   

                                           Description  Cluster Super_Category  
0    This is a PDF app for Android and the best vie...        3         reader  
1    1. The mobile health insurance card app is a p...        1        payment  
2    Our features include: View Multiple Types of F...        3         reader  
3    Adot, your own newly evolved AI personal assis...        4         vid